## MutiSearch RAG application

![Alt text](https://i.imgur.com/5IyyRl2.png)



- In LangChain, "tools" refer to specific functionalities or external resources that an AI model can use to accomplish tasks beyond its basic capabilities. 
- These tools can interact with different APIs, databases, web browsers, or even perform computations to handle more complex queries. 
- They extend the functionality of language models by enabling them to interact with external systems.

In [13]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [14]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [15]:
wiki.name

'wikipedia'

In [42]:
wiki.run("Data Science")

'Page: Data science\nSummary: Data science is an interdisciplinary academic field that uses statistics, scientific computing, scientific methods, processing, scientific visualization, algorithms and sys'

In [16]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Data ingestion
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

# Data Transformation
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

# Embedding
vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="llama3.2"))
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C32334BCD0>, search_kwargs={})

In [18]:
from langchain.tools.retriever import create_retriever_tool

# create_retriever_tool(retriever=retriever, name="langsmith_search", 
# description="Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="LangSmith Search",
    description="Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)

In [19]:
retriever_tool.name

'LangSmith Search'

In [71]:
retrieved_data = retriever_tool.run("LangSmith")
print(retrieved_data)


Get started with LangSmith | ğŸ¦œï¸�ğŸ› ï¸� LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmithâ€‹PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API keyâ€‹To create an API key head to the Settings page. Then click Create API Key.3. Set up your environmentâ€‹Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first traceâ€‹Tracing to LangSmith for LangChain usersThere is no need to use the LangSmith SD

In [20]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [44]:
arxiv.run("Paper 1234.578")

'Published: 2008-02-04\nTitle: This paper as been withdrawn\nAuthors: R. K. Zheng\nSummary: This paper as been withdrawn. This paper as been withdrawn. This paper as\nbeen withdrawn. This paper as been wit'

In [57]:
tools = [wiki, arxiv, retriever_tool]

In [53]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\shrey\\anaconda3\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='LangSmith Search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001C3638D7920>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vector

In [66]:
from langchain.tools import Tool

# Define the tools with wrapped functions
tools = [
    Tool(
        name="Wikipedia",
        func=lambda query: wiki.run(query),
        description="Retrieves information from Wikipedia"
    ),
    Tool(
        name="ArXiv",
        func=lambda query: arxiv.run(query),
        description="Retrieves information from ArXiv"
    ),
    Tool(
        name="Retriever",
        func=lambda query: retriever_tool.run(query),
        description="Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
    )
]

tools


[Tool(name='Wikipedia', description='Retrieves information from Wikipedia', func=<function <lambda> at 0x000001C32356F060>),
 Tool(name='ArXiv', description='Retrieves information from ArXiv', func=<function <lambda> at 0x000001C32356E840>),
 Tool(name='Retriever', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', func=<function <lambda> at 0x000001C32356EFC0>)]

- In LangChain, agents are an advanced feature that allow a language model to interact with external environments by dynamically calling tools or APIs based on user queries. 
- Unlike static models that simply generate responses, agents are designed to autonomously decide which actions or tools to use and in what sequence, depending on the context of the task.

In [63]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.2")
llm

Ollama(model='llama3.2')

In [75]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    llm=llm,
    tools=tools,
    agent_type="zero-shot-react-description",
    verbose=True,
    max_iterations=4  
)
agent

AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nWikipedia(query) - Retrieves information from Wikipedia\nArXiv(query) - Retrieves information from ArXiv\nRetriever(query) - Search for information about LangSmith. For any questions about LangSmith, you must use this tool!\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Wikipedia, ArXiv, Retriever]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input que

In [ ]:
try:
    result = agent.invoke({"input": "What is LangSmith?"}) 
    print(result)
      
except Exception as e:
    print(f"Error: {e}")


In [69]:
try:
    result = agent.invoke({"input":"What's the paper 1605.08386 about?"})  
    print(result)
except TypeError as e:
    print(f"Error: {e}")




> Entering new AgentExecutor chain...
Action: ArXiv
Action Input: 1605.08386
Observation: Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allo
Thought:Question: What's the paper 1605.08386 about?

Thought: I need to find more information about this paper.

Action: Wikipedia
Action Input: 1605.08386
Observation: Page: Lunar Saros 111
Summary: Saros cycle series 111 for lunar eclipses occurs at the moon's descending node, repeats every 18 years 11 and 1/3 days. It contains 71 events.
The first total lunar ecli
Thought:Question: What's the paper 1605.08386 about?

Thought: I need to find more information about this paper.

Action: Wikipedia
Action Input: 1605.08386
Observation: Page: Lunar Saros 111
Summary: Saros cycle series 111 for lunar eclipses occurs at the moon's descending node, repeats every 18 years 11 and 1/3 days. It contains 71 

In [70]:
try:
    result = agent.invoke({"input":"What is MAchine LEarning"})  
    print(result)
except TypeError as e:
    print(f"Error: {e}")



> Entering new AgentExecutor chain...
Question: What is Machine Learning

Thought: I need to understand what machine learning is, but I don't know where to start.

Action: Wikipedia
Action Input: "machine learning
Observation: Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and gen
Thought:Question: What is Machine Learning

Thought: Question: What is Machine Learning

Thought: I need to understand what machine learning is, but I don't know where to start.

Action: Wikipedia
Action Input: "machine learning
Observation: Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and gen
Thought:Question: What is Machine Learning

Thought: Question: What is Machine Learning

Thought: Question: What is Machine Lea